In [1]:
from pathlib import Path
import os
import pandas as pd
import cudf as gd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

print(gd.__version__)

0.7.2+0.g3ebd286.dirty


In [2]:
data_dir = Path("/media/wwymak/Storage/kaggle-nyc-taxi/")

In [3]:
!nvidia-smi

Thu Jul  4 23:32:44 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:04:00.0 Off |                  N/A |
| 27%   35C    P2    53W / 250W |    906MiB / 10989MiB |      5%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
GPU_MEMORY = 11

In [5]:
!wc -l {str(data_dir/"train.csv")}

55423856 /media/wwymak/Storage/kaggle-nyc-taxi/train.csv


In [6]:
!head {str(data_dir/"train.csv")}

In [14]:
%time train_df = gd.read_csv(str(data_dir/"train.csv"), usecols=["fare_amount",\
                          "pickup_datetime","pickup_longitude","pickup_latitude",\
                          "dropoff_longitude","dropoff_latitude","passenger_count"],\
           nrows=8000000)

CPU times: user 2.14 s, sys: 375 ms, total: 2.52 s
Wall time: 2.52 s


In [16]:
train_df["datestr"] = train_df.pickup_datetime.str.split(" ")[0]
train_df["timestr"] = train_df.pickup_datetime.str.split(" ")[1]

train_df["year"] = train_df.datestr.str.split("-")[0].astype("int32")
train_df["month"] = train_df.datestr.str.split("-")[1].astype("int32")
train_df["day"] = train_df.datestr.str.split("-")[2].astype("int32")
train_df["hour"] = train_df.timestr.str.split(":")[0].astype("int32")
train_df["minute"] = train_df.timestr.str.split(":")[1].astype("int32")
train_df["second"] = train_df.timestr.str.split(":")[2].astype("int32")

train_df = train_df.drop(["datestr", "timestr"], axis=1)

In [18]:
for col in train_df.columns:
    if train_df[col].dtype=="float64":
         train_df[col] = train_df[col].astype("float32")
    elif train_df[col].dtype=="int64":
         train_df[col] = train_df[col].astype("int32")

In [19]:
train_df.head().to_pandas()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,minute,second
0,4.5,2009-06-15 17:26:21 UTC,-73.844315,40.721317,-73.841614,40.712276,1,2009,6,15,17,26,21
1,16.9,2010-01-05 16:52:16 UTC,-74.016045,40.711304,-73.979271,40.782005,1,2010,1,5,16,52,16
2,5.7,2011-08-18 00:35:00 UTC,-73.982735,40.761269,-73.991241,40.750561,2,2011,8,18,0,35,0
3,7.7,2012-04-21 04:30:42 UTC,-73.987129,40.733143,-73.991570,40.758091,1,2012,4,21,4,30,42
4,5.3,2010-03-09 07:51:00 UTC,-73.968094,40.768009,-73.956657,40.783764,1,2010,3,9,7,51,0


In [38]:
for col in train_df.columns:
    print(col, train_df[col].null_count)

fare_amount 0
pickup_datetime 0
pickup_longitude 0
pickup_latitude 0
dropoff_longitude 49
dropoff_latitude 49
passenger_count 0
year 0
month 0
day 0
hour 0
minute 0
second 0


In [32]:
train_df[['fare_amount', 'pickup_longitude', 'pickup_latitude']].describe().to_pandas()

,stats,fare_amount,pickup_longitude,pickup_latitude
0,count,8.000000e+06,8.000000e+06,8.000000e+06
1,mean,1.133915e+01,-7.252072e+01,3.991624e+01
2,std,9.807189e+00,1.289849e+01,9.122808e+00
3,min,-1.077500e+02,-3.439245e+03,-3.492264e+03
4,25%,6.000000e+00,-7.399208e+01,4.073491e+01
5,50%,8.500000e+00,-7.398182e+01,4.075263e+01
6,75%,1.250000e+01,-7.396711e+01,4.076711e+01
7,max,1.273310e+03,3.439426e+03,3.344459e+03


In [33]:
train_df[['year', 'month', 'day', 'hour', 'minute', 'second']].describe().to_pandas()



,stats,year,month,day,hour,minute,second
0,count,8.000000e+06,8.000000e+06,8.000000e+06,8.000000e+06,8.000000e+06,8.000000e+06
1,mean,-1.357441e+02,6.265835e+00,1.571797e+01,1.351102e+01,2.957339e+01,1.582983e+01
2,std,1.860274e+00,3.435541e+00,8.684377e+00,6.516063e+00,1.732643e+01,1.943344e+01
3,min,2.009000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,25%,2.010000e+03,3.000000e+00,8.000000e+00,9.000000e+00,1.500000e+01,0.000000e+00
5,50%,2.012000e+03,6.000000e+00,1.600000e+01,1.400000e+01,3.000000e+01,4.000000e+00
6,75%,2.013000e+03,9.000000e+00,2.300000e+01,1.900000e+01,4.500000e+01,3.200000e+01
7,max,2.015000e+03,1.200000e+01,3.100000e+01,2.300000e+01,5.900000e+01,5.900000e+01


NOTE cudf does not handle nulls in dataframe.describe...

In [41]:
for col in ['dropoff_latitude', 'dropoff_longitude']:
    train_df[col] = train_df[col].fillna(-800)

In [43]:
train_df[['dropoff_latitude', 'dropoff_longitude']].describe().to_pandas()

,stats,dropoff_latitude,dropoff_longitude
0,count,8.000000e+06,8.000000e+06
1,mean,3.990910e+01,-7.252472e+01
2,std,9.540772e+00,1.298512e+01
3,min,-3.488080e+03,-3.412653e+03
4,25%,4.073404e+01,-7.399139e+01
5,50%,4.075316e+01,-7.398016e+01
6,75%,4.076811e+01,-7.396368e+01
7,max,3.351403e+03,3.457622e+03
